In [72]:
# coding=utf8
import numpy as np
import pandas as pd
import json

In [22]:
data_file = open('after.csv')
data = pd.read_csv(data_file)

In [23]:
data

,ab_id,batter_id,event,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015,572761,A,top,1,0,L,452657,L,True
1,2015,518792,B,top,1,0,L,452657,L,True
2,2015,407812,A,top,1,0,L,452657,R,True
3,2015,425509,A,top,2,0,L,452657,R,True
4,2015,571431,A,top,3,0,L,452657,L,True
5,2015,451594,B,top,0,1,R,425794,L,False
6,2015,624585,A,top,1,1,R,425794,R,False
7,2015,519203,A,top,2,1,R,425794,L,False
8,2015,516770,A,top,3,1,R,425794,R,False
9,2015,425877,A,top,1,0,L,452657,R,True


In [24]:
def createC1( dataSet ):
    C1 = []
    for transaction in np.array(dataSet):
        for item in transaction:
            if [item] not in C1:
                C1.append( [item] )
    C1.sort()
    return list(map( frozenset, C1 ))

def scanD( D, Ck, minSupport ):
    ssCnt = {}
    for tid in D:
        # 对于每一条transaction
        if Ck is not None:
            for can in Ck:
                # 对于每一个候选项集can，检查是否是transaction的一部分
                # 即该候选can是否得到transaction的支持
                if can.issubset( tid ):
                    ssCnt[ can ] = ssCnt.get( can, 0) + 1
    numItems = float( len( D ) )
    retList = []
    supportData = {}
    for key in ssCnt:
        # 每个项集的支持度
        support = ssCnt[ key ] / numItems
        
        # 将满足最小支持度的项集，加入retList
        if support >= minSupport:
            retList.insert( 0, key )
            
            # 汇总支持度数据
            supportData[ key ] = support
    return retList, supportData


In [25]:
def aprioriGen( Lk, k ):
    retList = []
    lenLk = len( Lk )
    for i in range( lenLk ):
        for j in range( i + 1, lenLk ):
            L1 = list( Lk[ i ] )[ : k - 2 ];
            L2 = list( Lk[ j ] )[ : k - 2 ];
            L1.sort();L2.sort()    
            if L1==L2:
                retList.append( Lk[ i ] | Lk[ j ] ) 
    return retList

def apriori( dataSet, minSupport = 0.5 ):
    C1 = createC1( dataSet )
    D =list( map( set, dataSet ))
    L1, suppData = scanD( D, C1, minSupport )
    L = [ L1 ]
    k = 2
    
    while ( len( L[ k - 2 ] ) > 0 ):
        Ck = aprioriGen( L[ k - 2 ], k )
        Lk, supK = scanD( D, Ck, minSupport )
        suppData.update( supK )
        L.append( Lk )
        k += 1
    return L, suppData

In [26]:
data = list(map(set, np.array(data)))

In [27]:
data

[{0, 1, 2015, 452657, 572761, 'A', 'L', 'top'},
 {0, 1, 2015, 452657, 518792, 'B', 'L', 'top'},
 {0, 1, 2015, 407812, 452657, 'A', 'L', 'R', 'top'},
 {0, True, 2, 2015, 425509, 452657, 'A', 'L', 'R', 'top'},
 {0, True, 3, 2015, 452657, 571431, 'A', 'L', 'top'},
 {0, 1, 2015, 425794, 451594, 'B', 'L', 'R', 'top'},
 {False, 1, 2015, 425794, 624585, 'A', 'R', 'top'},
 {False, 1, 2, 2015, 425794, 519203, 'A', 'L', 'R', 'top'},
 {False, 1, 3, 2015, 425794, 516770, 'A', 'R', 'top'},
 {0, 1, 2015, 425877, 452657, 'A', 'L', 'R', 'top'},
 {0, 1, 2015, 452657, 543939, 'B', 'L', 'top'},
 {0, 1, 2015, 445055, 452657, 'A', 'L', 'top'},
 {0, True, 2, 2015, 425794, 452657, 'A', 'L', 'R', 'top'},
 {0, True, 2, 2015, 452657, 572761, 'A', 'L', 'top'},
 {0, True, 3, 2015, 452657, 518792, 'C', 'L', 'top'},
 {0, 2, 2015, 425794, 458085, 'B', 'L', 'R', 'top'},
 {False, 1, 2, 2015, 425794, 592609, 'A', 'R', 'top'},
 {False, 2, 2015, 424325, 425794, 'B', 'R', 'top'},
 {False, 2, 3, 2015, 425794, 452657, 'A', 

## 找出频繁项集，将最小支持度设置为0.1

In [31]:
L, suppData = apriori(data, 0.1)

In [32]:
L

[[frozenset({'B'}),
  frozenset({2018}),
  frozenset({2015}),
  frozenset({'L'}),
  frozenset({0}),
  frozenset({'R'}),
  frozenset({2016}),
  frozenset({'bot'}),
  frozenset({1}),
  frozenset({2017}),
  frozenset({'A'}),
  frozenset({2}),
  frozenset({'top'}),
  frozenset({'mid'}),
  frozenset({3})],
 [frozenset({2015, 'R'}),
  frozenset({1, 2015}),
  frozenset({2017, 'R'}),
  frozenset({2016, 'L'}),
  frozenset({0, 2018}),
  frozenset({1, 'mid'}),
  frozenset({2, 2018}),
  frozenset({'A', 'bot'}),
  frozenset({2018, 'A'}),
  frozenset({2018, 'B'}),
  frozenset({2017, 'L'}),
  frozenset({1, 2}),
  frozenset({0, 2016}),
  frozenset({2, 'A'}),
  frozenset({2, 'B'}),
  frozenset({2015, 'A'}),
  frozenset({2018, 'L'}),
  frozenset({1, 2018}),
  frozenset({0, 'mid'}),
  frozenset({0, 'bot'}),
  frozenset({2, 'top'}),
  frozenset({2, 2015}),
  frozenset({0, 'L'}),
  frozenset({0, 1}),
  frozenset({'L', 'R'}),
  frozenset({2016, 'R'}),
  frozenset({'B', 'mid'}),
  frozenset({'A', 'mid'}),
  

## 导出规则,设置信度为0.8


In [43]:
def createCk(Lk, k):
    Ck=[]
    length=len(Lk)
    for i in range(length):
        for j in range(i+1,length):
            L1=list(Lk[i])[:k-2]
            L2=list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                Ck.append((Lk[i]|Lk[j]))
    return Ck
def generateRules(L, supportRate, minConf=0.5):
    bigRuleList = []
    length=len(L)
    for i in range(1,length):
        for freqSet in L[i]:
            H1=[frozenset([item]) for item in freqSet]
            if i>1:
                rulesFromConseq(freqSet, H1, supportRate, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportRate, bigRuleList, minConf)
    return bigRuleList

def calcConf(freqSet, H1, supportRate, bigRuleList, minConf):
    prunedH = []
    for conseq in H1:
        conf = supportRate[freqSet]/supportRate[freqSet-conseq]
        if conf>=minConf:
            print freqSet-conseq, "---->", conseq, ":", json.dumps([supportRate[freqSet], conf])
            bigRuleList.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H1, supportRate, bigRuleList, minConf):
    length=len(H1[0])
    if len(freqSet)>length+1:
        newH=createCk(H1,length+1)
        newH=calcConf(freqSet, newH, supportRate, bigRuleList, minConf)
        if len(newH)>1:
            rulesFromConseq(freqSet, newH, supportRate, bigRuleList, minConf)

In [44]:
rule = generateRules(L, suppData, 0.8)

frozenset([2015]) ----> frozenset(['R']) : [0.22873381425169742, 0.9198064274347291]
frozenset([2017]) ----> frozenset(['R']) : [0.23281815370028458, 0.9282898563212201]
frozenset(['L']) ----> frozenset(['R']) : [0.5315205925533739, 0.8753539501166672]
frozenset([2016]) ----> frozenset(['R']) : [0.23175790023892845, 0.9274888381998422]
frozenset(['mid']) ----> frozenset(['R']) : [0.3125883825934745, 0.9257998447913083]
frozenset(['top']) ----> frozenset([0]) : [0.2944560224422567, 0.878775908354361]
frozenset([0]) ----> frozenset(['R']) : [0.6590238374692222, 0.9262752244812726]
frozenset([2]) ----> frozenset(['R']) : [0.3836604811794881, 0.9248215035699286]
frozenset([2018]) ----> frozenset(['R']) : [0.23100424236448677, 0.921643118109218]
frozenset(['B']) ----> frozenset(['R']) : [0.38051618811192495, 0.9322356382503499]
frozenset(['A']) ----> frozenset(['R']) : [0.497387184304467, 0.9187779898456895]
frozenset([3]) ----> frozenset(['R']) : [0.2886671736073875, 0.921610142084043]
fro

## 使用全自信度和lift对规则进行评价


In [67]:
def select_conf(L,suppData,thredhold=0.8):
    confList = []
    for itemSet in L:
            for record in itemSet:
                if(len(record)>1): 
                    for i in record: 
                        tar = suppData[frozenset([i])]
                        bset = record - frozenset([i])
                        aset = suppData[record]
                        ret = suppData.get(bset)
                        try:
                            if suppData.get(bset):
                                con = aset/ret
                                if con > thredhold:
                                    confList.append([frozenset([i]),record,con])
                        except TypeError as e:
                            print(e)

    return confList


In [69]:
confList = select_conf(L,suppData)
confList

[[frozenset({'R'}), frozenset({2015, 'R'}), 0.9198064274347291],
 [frozenset({'R'}), frozenset({2017, 'R'}), 0.9282898563212201],
 [frozenset({'R'}), frozenset({'L', 'R'}), 0.8753539501166672],
 [frozenset({'R'}), frozenset({2016, 'R'}), 0.9274888381998422],
 [frozenset({'R'}), frozenset({'R', 'mid'}), 0.9257998447913083],
 [frozenset({0}), frozenset({0, 'top'}), 0.878775908354361],
 [frozenset({'R'}), frozenset({0, 'R'}), 0.9262752244812726],
 [frozenset({'R'}), frozenset({2, 'R'}), 0.9248215035699286],
 [frozenset({'R'}), frozenset({2018, 'R'}), 0.921643118109218],
 [frozenset({'R'}), frozenset({'B', 'R'}), 0.9322356382503499],
 [frozenset({'R'}), frozenset({'A', 'R'}), 0.9187779898456895],
 [frozenset({'R'}), frozenset({3, 'R'}), 0.921610142084043],
 [frozenset({'R'}), frozenset({'R', 'top'}), 0.9396499600944833],
 [frozenset({'R'}), frozenset({'R', 'bot'}), 0.9070803946896009],
 [frozenset({'R'}), frozenset({1, 'R'}), 0.9243357660203021],
 [frozenset({'R'}), frozenset({0, 'R', 'mid

In [74]:
def all_confidence(conList,suppData):
    result = []
    for item in conList:
        score = suppData.get(item[1])/max(suppData.get(item[0]),suppData.get(item[1]-item[0]))
        result.append(['all confidence:{0}'.format(score),item])
    return result

In [75]:
all_confidence(confList,suppData)

[['all confidence:0.247463293744',
  [frozenset({'R'}), frozenset({2015, 'R'}), 0.9198064274347291]],
 ['all confidence:0.251882072384',
  [frozenset({'R'}), frozenset({2017, 'R'}), 0.9282898563212201]],
 ['all confidence:0.575043252595',
  [frozenset({'R'}), frozenset({'L', 'R'}), 0.8753539501166672]],
 ['all confidence:0.25073500187',
  [frozenset({'R'}), frozenset({2016, 'R'}), 0.9274888381998422]],
 ['all confidence:0.338184150846',
  [frozenset({'R'}), frozenset({'R', 'mid'}), 0.9257998447913083]],
 ['all confidence:0.413865633958',
  [frozenset({0}), frozenset({0, 'top'}), 0.878775908354361]],
 ['all confidence:0.712986883943',
  [frozenset({'R'}), frozenset({0, 'R'}), 0.9262752244812726]],
 ['all confidence:0.41507586739',
  [frozenset({'R'}), frozenset({2, 'R'}), 0.9248215035699286]],
 ['all confidence:0.249919632002',
  [frozenset({'R'}), frozenset({2018, 'R'}), 0.921643118109218]],
 ['all confidence:0.41167410923',
  [frozenset({'R'}), frozenset({'B', 'R'}), 0.932235638250349

In [59]:
def liftCheck(conList,suppData):
    liftList_true=[]
    liftList_no=[]
    liftList_false=[]
    for item in conList:
        score = suppData.get(item[1])/(suppData.get(item[0])*suppData.get(item[1]-item[0]))
        if score>1:
            liftList_true.append(item)
        if score<1:
            liftList_false.append(item)
        if score ==1:
            liftList_no.append(item)
    return liftList_true,liftList_no,liftList_false

In [60]:
liftCheck(conList, suppData)

([[frozenset({'R'}), frozenset({2017, 'R'}), 0.9282898563212201],
  [frozenset({'R'}), frozenset({2016, 'R'}), 0.9274888381998422],
  [frozenset({'R'}), frozenset({'R', 'mid'}), 0.9257998447913083],
  [frozenset({0}), frozenset({0, 'top'}), 0.878775908354361],
  [frozenset({'R'}), frozenset({0, 'R'}), 0.9262752244812726],
  [frozenset({'R'}), frozenset({2, 'R'}), 0.9248215035699286],
  [frozenset({'R'}), frozenset({'B', 'R'}), 0.9322356382503499],
  [frozenset({'R'}), frozenset({'R', 'top'}), 0.9396499600944833],
  [frozenset({'R'}), frozenset({1, 'R'}), 0.9243357660203021],
  [frozenset({'R'}), frozenset({0, 'R', 'mid'}), 0.9259760816039394],
  [frozenset({'R'}), frozenset({2, 'B', 'R'}), 0.9305811553791138],
  [frozenset({'R'}), frozenset({'B', 'R', 'top'}), 0.9467708292337361],
  [frozenset({'R'}), frozenset({0, 2016, 'R'}), 0.9300209348879143],
  [frozenset({0}), frozenset({0, 'R', 'top'}), 0.8785015057010733],
  [frozenset({'R'}), frozenset({0, 'R', 'top'}), 0.9393565491807789],
 